In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
import nibabel as nib

# Define the 3D CNN model
def create_3d_cnn(input_shape):
    model = Sequential()
    
    # First convolutional layer
    model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Second convolutional layer
    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Third convolutional layer
    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Flatten the output and add dense layers
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer for regression
    model.add(Dense(1, activation='linear'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

# Example usage
input_shape = (260, 311, 260, 1)  # Example input shape (e.g., 64x64x64 voxel data with 1 channel)

# Assuming you have your data loaded in X_train and y_train
# X_train = ... (your 3D data)
# y_train = ... (your regression targets)

# Train the model
# model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

2025-04-10 18:14:16.680166: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import numpy as np

# Load the NIfTI image
directory = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/'
files = os.listdir(directory)
files2 = []
for i in files:
    if 'nii' in i:
        files2.append(i)

image_dict = {}
count = 0
for j in files2:
    if j != '.nii.gz' and count < 100:
        path1 = directory + j
        nifti_image = nib.load(path1)
        image_data = nifti_image.get_fdata()
        max_value = np.max(image_data)
        image_data = image_data / max_value
        image_dict[j[0:6]] = image_data
        count += 1

#path1 = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/100206_T1w.nii.gz'

#print(shapes)
nifti_image = nib.load(path1)

# Get the data from the NIfTI image
image_data = nifti_image.get_fdata()
print("Shape of the NIfTI image:", image_data.shape)

# Select a slice to display (e.g., the middle slice)
slice_index = image_data.shape[2] // 2
slice_data = image_data[:, :, slice_index]

# Display the slice
#plt.imshow(slice_data.T, cmap='gray', origin='lower')
#plt.title('Middle Slice of NIfTI Image')
#plt.axis('off')
#plt.show()


Shape of the NIfTI image: (260, 311, 260)


In [3]:
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/age/ages.csv')

# Display the first few rows of the DataFrame
#print(df.head())
#print(type(df['age']))
ages = list(df['age'])
ids = list(df['subject_id'])
key = {}
for i,j in zip(ids,ages):
    key[i] = j
#print(key)
#print("9")
#print(key.get(100004))

In [ ]:
input_shape = (260, 311, 260, 1)  # Example input shape (e.g., 64x64x64 voxel data with 1 channel)
model = create_3d_cnn(input_shape)
# Print the model summary
#model.summary()
x_train = []
y_train = []
for i in df['subject_id']:
    item = image_dict.get(str(i))
    if item is not None:
        x_train.append(item)
        y_train.append(key.get(i))

num_epochs = 5
print(y_train)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
print(y_train)
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    
    history = model.fit(
        x_train, y_train,
        epochs=1,
        batch_size=2,
        validation_split=0.2
    )